# Keras Callback

Dans ce notebook nous allons ajouter un callback EarlyStopping qui arrete l'entraînement quand le gain de performance stagne.

On va appliquer ce callback sur le loss pendant l'etape de fit()




In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

print(tf.__version__)

## Le dataset CIFAR-10

https://www.cs.toronto.edu/~kriz/cifar.html

Une collection de 60k images en couleur de petite taille 32x32 classées en 10 catégories. chaque catégorie contient 6000 images.



- airplane
- automobile
- bird
- cat
- deer
- dog
- frog
- horse
- ship
- truck

In [ ]:
# charger le dataset à partir de Keras
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

In [ ]:
# verifier la présence des images
! ls -al /root/.keras/datasets/

In [ ]:
# Afficher une image
idx = 88

print(f'LABEL: {train_labels[idx]} ')
fig, ax = plt.subplots(1,1, figsize = (1,1))
plt.imshow(train_images[idx])
plt.axis('off')
plt.show()

# le modele

In [ ]:
## construire le modele et le compiler

'''
-- ecrire une fonction create_model qui prend n_nodes et lr en entree
et qui crée et compile un model sequential avec les couches Flatten, Dense : activation relu, et Dense avec activation softmax
la fonction retourne le modele compilé
'''
def create_model(n_nodes = 256, lr = 0.01):
    model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(32, 32, 3)),
    tf.keras.layers.Dense(n_nodes, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])

    # Compile the model
    model.compile(optimizer=tf.optimizers.Adam(  learning_rate=lr),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    return model


N'hesitez pas a changer les parametres
- n_nodes = 128, 1024
- lr = 0.2, 0.000001, etc ...
etc

In [ ]:
model = create_model(n_nodes = 256, lr = 0.01 )

In [ ]:
model.summary()

Rajoutons maintenant le callback early_stopping dans la phase d'entraînement

https://keras.io/api/callbacks/early_stopping/

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3, verbose = 1, min_delta = 0.001)

Notez les functions du callback :

- quand: on_epoch_end, on_epoch_begin
- le meilleur modele : restore_best_weights
- faire un minimum d'iterations : start_from_epoch

In [ ]:
early_stopping


In [ ]:
# entrainer le modele avec beaucoup d'epochs

history = model.fit(train_images, train_labels, epochs=25, callbacks=[early_stopping])

In [ ]:
# acceder aux courbes accuracy et loss
history.history['accuracy']

## reduire le learning rate

Ajoutons un autre callback qui réduit le learning rate quand la valeur loss stagne

https://keras.io/api/callbacks/reduce_lr_on_plateau/


### questions

- Pourquoi réduire le learnign rate ?
- Pourquoi ne pas simplement fixer un learning rate bas des le depart ?
- Comment gérer les 2 patiences entre les 2 callbacks ? L'une doit elle etre necessairement inferieure a l'autre ?

In [ ]:
# reset le modele
model = create_model(n_nodes = 128, lr = 0.02 )

# callbacks
# early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4, verbose = 1)
# rajouter le call back ReduceLROnPlateau
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2, patience=2, min_lr=0.001, verbose = 1)


history = model.fit(train_images, train_labels, epochs=25, callbacks=[early_stopping, reduce_lr])

### Qu'observez vous ?

- learning rate
- loss
- nombre d'epochs




### Visualisez

Afficher dans une figure l'evolution du loss, du learning et de l'accuracy

In [ ]:
fig, ax = plt.subplots(3,1, figsize = (8,8))
plt.subplot(3,1,1)
plt.plot(history.history['accuracy'][1:], label = 'accuracy')
plt.subplot(3,1,2)
plt.xlabel('accuracy')
plt.grid(visible = True)
plt.plot(history.history['loss'][1:], label = 'loss')
plt.xlabel('loss')
plt.grid(visible = True)
plt.subplot(3,1,3)
plt.plot(history.history['lr'][1:], label = 'learning_rate')
plt.xlabel('lr')
plt.grid(visible = True)
plt.tight_layout()